In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/fake-news/train.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.describe

In [ ]:
train_df.dropna(inplace=True)

In [ ]:
X = train_df.drop('label', axis=1)

In [ ]:
y = train_df['label']

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.preprocessing.text import one_hot

In [ ]:
#Vocabulary -- dictionary of 5000 words to get all sentences
voc_size = 5000

# One Hot Representation

In [ ]:
message = X.copy()

In [ ]:
message = message.reset_index() # to set the missing index for null values

In [ ]:
import re
import nltk
from nltk.corpus import stopwords

# Dataset Pre-processing

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer

In [ ]:
def preprocessing(column_data):
    ps = PorterStemmer()
    corpus = []
    for i in range(len(column_data)):
        print(i)
        review = re.sub('[^a-zA-Z]',' ', column_data[i])
        review = review.lower()
        review = review.split()
        review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)
    return corpus

In [ ]:
preprocessed_title = preprocessing(message['title'])

In [ ]:
preprocessed_title[30]

In [ ]:
one_hot_repr_title = [one_hot(words,voc_size) for words in preprocessed_title]

In [ ]:
one_hot_repr_title

# Padding Sequences for tilte column

In [ ]:
sent_length = 20

In [ ]:
embedded_title = pad_sequences(one_hot_repr_title, padding='pre', maxlen=sent_length)

In [ ]:
embedded_title

In [ ]:
embedded_test_title = pad_sequences(one_hot_repr_title, padding='pre', maxlen=sent_length)

In [ ]:
# Creating Vectors

In [ ]:
embedded_vec_features = 40

# Model Building

In [ ]:
from tensorflow.keras.layers import Dropout
model= Sequential()
model.add(Embedding(voc_size,embedded_vec_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
X_final = np.array(embedded_title)
y_final = np.array(y)

In [ ]:
X_final[0]

In [ ]:
test = np.array(embedded_test_title)

In [ ]:
X_final.shape, y_final.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.15,random_state=42)

In [ ]:
history=model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=15, batch_size=32)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()
plt.show()
plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

In [ ]:
import joblib
joblib.dump(preprocessing, "data_transformer.joblib")
model.save("price_prediction_model.h5")

In [ ]:
preprocessing

# Test Dataset

In [ ]:
test_df = pd.read_csv('/kaggle/input/fake-news/test.csv')

In [ ]:
test_df.isnull().sum()

In [ ]:
test_df.fillna('No data here', inplace=True)

In [ ]:
test_df.head()

In [ ]:
preprocessed_test_title = preprocessing(test_df['title'])

In [ ]:
one_hot_repr_test_title = [one_hot(words,voc_size) for words in preprocessed_test_title]

In [ ]:
embedded_test_title = pad_sequences(one_hot_repr_test_title, padding='pre', maxlen=sent_length)

In [ ]:
testing_array = np.array(embedded_test_title)

In [ ]:
y_pred = model.predict_classes(testing_array)

In [ ]:
y_pred

In [ ]:
y_pred.shape

In [ ]:
label = [i[0] for i in y_pred]

In [ ]:
id = test_df['id']

In [ ]:
submission.to_csv('submit.csv', index=False)